In [1]:
import sys
sys.path.append("..")

In [2]:
import examples
import fseof_additions
import fseof_media
import dh5_alpha

#load model
model = dh5_alpha.ecoli_dh5a()

#add necessary genes (lycopene pathways)
fseof_additions.add_crtE(model)
fseof_additions.add_crtB(model)
fseof_additions.add_crtI(model)
fseof_additions.add_lyco_dem(model)

#set LB media
fseof_media.simulate_LB_media(model)





In [3]:
#check objective to see if matches the one presented in paper

#max theoretical lycopene flux
model.objective = "LYCOdem"
solution = model.optimize()
solution.objective_value


0.9657492354740086

In [ ]:
#run fseof to verify/cross-check with choi et al. paper
from cobra_fseof.fseof import fseof
results = fseof(model, 10, "LYCOdem", "BIOMASS_Ec_iJO1366_core_53p95M", "max")
print(results.fva)

In [9]:
#export more/different results if needed
results.scan.to_csv('targetlist.csv')
results.fva.to_csv('targetfvalist.csv')
